# Notebook for loading excel files

### Function that converts .xlsx files to .xls 

In [2]:
import pyexcel as p

def xlsx_to_xls(filename):
    """Gets the filename of the excel file you want to chenge the format from xlsx to xls.

    Parameters
    ----------
    filename : str
        The complete path of the file, the format of which you want to modify.
        The file must be in the format xlsx.

    Returns
    -------
    None
        Creates a new file and saves it in the same directory with the orginal file,
        but with format xls.

    """
    if not filename.endswith('.xlsx'):
        raise Exception('Invalid File Format')

    else:
        dest_filename = filename.replace('.xlsx', '.xls')
        print(dest_filename)
        p.save_as(file_name = filename, dest_file_name = dest_filename)
    



xlsx_to_xls('/home/billys/Documents/thesis-bill/DATA/ENERSHARE_DATA _NTUA/Flows 2017 - Hourly- Excel/Hourly-Flows-02_2017-1.xlsx')

/home/billys/Documents/thesis-bill/DATA/ENERSHARE_DATA _NTUA/Flows 2017 - Hourly- Excel/Hourly-Flows-02_2017-1.xls


KeyboardInterrupt: 

### Functions for adding holidays and weekdays in our dataframe 

In [7]:
from datetime import date
import datetime
import holidays


'''
def merge_date_and_time(df, date):
    if type(df['DateTime']) is datetime.datetime:
        return datetime.datetime.combine(date, df['DateTime'].time())
Function that does not work for now'''   


def holiday_func(df):
    """Gets a df, checks the date of a given row and returns True
        if the date belongs in holiday based on country

    Parameters
    ----------
    df : DataFrame     

    Returns
    -------
    Boolean
        Returns True if the datetime is a holiday based on the country or False if it is not.
    """
    if df['DateTime'] in holidays.country_holidays('GR'):
        return True
    else:
        return False
    
    
def weekday_func(df):
    """Gets a dataframe and adds a column that has the weekday of a given row

    Parameters
    ----------
    df : DataFrame     

    Returns
    -------
    None
    """
    return df['DateTime'].weekday()

### Function that reads a single excel file

The function below reads an excel file by its name, modifies its format and
merges all the sheets in a single dataframe

In [8]:
import os
import pandas as pd
import datetime
import warnings


def read_excel_file(filename):
    """Gets a filename of an excel file that has an xls format and returns
        a merged dataframe with all sheets together

    Parameters
    ----------
    filename : str
        The full path of the excel file

    Returns
    -------
    DataFrame
        A dataframe that has all the sheets of the excel file in the format needed
    """
    warnings.filterwarnings('ignore')

    dfs_dict = pd.read_excel(filename, sheet_name=None, engine='xlrd')
    print("The number of sheets in this excel file is", len(dfs_dict))
    print(dfs_dict.keys())
    prev_df = pd.DataFrame()
    result = pd.DataFrame()
    for df in dfs_dict.values():
        df.columns = df.iloc[1]
        df = df.rename(columns={'Time':'DateTime'})

        
        for i in range(2, len(df['DateTime'])):
            if type(df['DateTime'][i]) is datetime.datetime:
                df['DateTime'][i] = datetime.datetime.combine(df['DateTime'][0].date(), df['DateTime'][i].time())
        
        # A line of code that does not work for now
        #df['DateTime'] = df.apply(lambda temp_df: merge_date_and_time(temp_df, temp_df['DateTime'][0].date()), axis=1)
        df = df.iloc[2:26, 1:45]
        df = df.dropna(axis=1)
        
        result = pd.concat([prev_df, df], ignore_index=True)
        prev_df = result

    result = result.rename(columns={'Time':'DateTime'})
    result['Holiday'] = result.apply(lambda temp_df: holiday_func(temp_df), axis=1)
    result['Weekday'] = result.apply(lambda temp_df: weekday_func(temp_df), axis=1)
    return result
    
read_excel_file('/home/billys/Documents/thesis-bill/DATA/ENERSHARE_DATA _NTUA/Flows 2015 - Hourly - Excel/Hourly-Flows-01_2015.xls')
    

The number of sheets in this excel file is 31
dict_keys(['01.01.2015', '02.01.2015', '03.01.2015', '04.01.2015', '05.01.2015', '06.01.2015', '07.01.2015', '08.01.2015', '09.01.2015', '10.01.2015', '11.01.2015', '12.01.2015', '13.01.2015', '14.01.2015', '15.01.2015', '16.01.2015', '17.01.2015', '18.01.2015', '19.01.2015', '20.01.2015', '21.01.2015', '22.01.2015', '23.01.2015', '24.01.2015', '25.01.2015', '26.01.2015', '27.01.2015', '28.01.2015', '29.01.2015', '30.01.2015', '31.01.2015'])


1,DateTime,AGIA TRIADA,SIDIROKASTRO,KIPI,AGIOI THEODOROI,ALOYMINION,ALOYMINION II,ALOYMINION III,ATHENS,ALEXANDROUPOLIS,...,MOTOR OIL II,XANTHI,OINOFYTA,PLATY,SALFA ANTHOUSSA,SALFA ANO LIOSSIA,SERRES,TRIKALA,Holiday,Weekday
0,2015-01-01 09:00:00,1.86764,2364.74,782.328,0.136442,435.095,0,90.9743,1022.78,8.8751,...,0,3.11111,32.4378,22.0459,5.06585,6.07902,43.2,36.4957,True,3
1,2015-01-01 10:00:00,0,2397.38,512.001,0.137161,433.904,0,95.3129,964.889,8.44497,...,0,0.497778,33.9043,24.1939,3.63352,4.36022,42.8444,41.0441,True,3
2,2015-01-01 11:00:00,219.046,2438.46,513.324,0.0921605,433.683,0,95.1789,909.655,7.99946,...,0,2.18667,33.7282,23.5737,0.0480136,0.0576163,43.68,38.4149,True,3
3,2015-01-01 12:00:00,738.346,2438.65,513.631,0.0933327,434.947,0,90.6915,946.129,144.839,...,0,0.604444,33.2008,22.2877,1.70794,2.04953,44.3022,35.5121,True,3
4,2015-01-01 13:00:00,715.792,2462.36,512.974,0.146365,436.569,0,90.0832,1090.82,7.07088,...,0,2.61333,33.894,20.1344,2.7108,3.25296,44.6578,34.5842,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2015-01-31 04:00:00,1768.63,1629.47,0,0.113794,422.186,0,82.3101,241.832,4.38386,...,0,2.64889,109.256,12.888,1.26389,1.51667,34.1333,9.19587,False,5
740,2015-01-31 05:00:00,1775.48,1639.24,0,1.10098,421.428,0,83.9954,240.088,4.87452,...,0,3.32444,93.9685,13.1835,0.405599,0.486719,31.6444,9.10032,False,5
741,2015-01-31 06:00:00,1768.13,1659.02,0,2.59893,421.301,0,82.2137,256.186,4.95547,...,0,2.88,88.4709,12.9348,6.76081,8.11297,29.1556,10.2563,False,5
742,2015-01-31 07:00:00,1782.08,1671.58,0,0.992284,421.592,0,83.1517,369.37,4.54345,...,0,2.79111,92.2171,12.9419,4.81906,5.78287,26.9511,11.6964,False,5


### Function that reads all excel files in multiple folders and merges them into a single dataframe and saves it into a csv file

In [10]:
import os
import pandas as pd
import datetime
import warnings


def read_files(folder_path, prefix):
    """Gets as input a directory that contains all excel files
    along with a prefix that every single file folder starts with and
    returns a dataframe that contains all data of all files. It also saves
    the dataframe into a csv file

    Parameters
    ----------
    folder_path : str
        The full path to the directory that contains other directories
        that contain the files we want to load
    
    prefix : str
        A prefix that exists in the beginning of every folder name

    Returns
    -------
    DataFrame
        A dataframe that contains all data of all excel files merged into a single one dataframe
    """

    
    warnings.filterwarnings('ignore')
    
    prev_df = pd.DataFrame()
    result = pd.DataFrame()
    for foldername in os.listdir(folder_path):
        if foldername.startswith(prefix):
            file_path = folder_path
            file_path = os.path.join(folder_path, foldername)
            # Open all excel files from those folders
            print(os.listdir(file_path))
            for filename in os.listdir(file_path):
                temp_filename = os.path.join(file_path, filename)
                if temp_filename.endswith('.xls'):
                    print(temp_filename)
                    dfs_dict = pd.read_excel(temp_filename, engine='xlrd', sheet_name=None, nrows=28)
                    for df in dfs_dict.values():
                        df.columns = df.iloc[1]

                        for i in range(2, len(df['Time'])):
                            if type(df['Time'][i]) is datetime.datetime:
                                df['Time'][i] = datetime.datetime.combine(df['Time'][0].date(), df['Time'][i].time())
                        df = df.iloc[2:26, 1:45]
                        df = df.dropna(axis=1)

                        result = pd.concat([prev_df, df], ignore_index=True)
                        prev_df = result
            #print(result)
    result.sort_values(by='Time', inplace=True)
    result = result.rename(columns={'Time':'DateTime'})
    result['Holiday'] = result.apply(lambda temp_df: holiday_func(temp_df), axis=1)
    result['Weekday'] = result.apply(lambda temp_df: weekday_func(temp_df), axis=1)
    result.to_csv('../EXCEL_TO_CSV/all_data.csv')
    return result
    
read_files('../DATA/ENERSHARE_DATA _NTUA', 'Flows')
    

['Hourly-Flows-05_2016.xls', 'Hourly-Flows-09_2016-1.xls', 'Hourly-Flows-04_2016.xls', 'Hourly-Flows-01_2016.xls', 'Hourly-Flows-12_2016.xls', 'Hourly-Flows-02_2016-1.xls', 'Hourly-Flows-06_2016-2.xls', 'Hourly-Flows-03_2016.xls', 'Hourly-Flows-08_2016.xls', 'Hourly-Flows-10_2016.xls', 'Hourly-Flows-07_2016.xls']
../DATA/ENERSHARE_DATA _NTUA/Flows 2016 - Hourly - Excel/Hourly-Flows-05_2016.xls
../DATA/ENERSHARE_DATA _NTUA/Flows 2016 - Hourly - Excel/Hourly-Flows-09_2016-1.xls
../DATA/ENERSHARE_DATA _NTUA/Flows 2016 - Hourly - Excel/Hourly-Flows-04_2016.xls
../DATA/ENERSHARE_DATA _NTUA/Flows 2016 - Hourly - Excel/Hourly-Flows-01_2016.xls
../DATA/ENERSHARE_DATA _NTUA/Flows 2016 - Hourly - Excel/Hourly-Flows-12_2016.xls
../DATA/ENERSHARE_DATA _NTUA/Flows 2016 - Hourly - Excel/Hourly-Flows-02_2016-1.xls
../DATA/ENERSHARE_DATA _NTUA/Flows 2016 - Hourly - Excel/Hourly-Flows-06_2016-2.xls
../DATA/ENERSHARE_DATA _NTUA/Flows 2016 - Hourly - Excel/Hourly-Flows-03_2016.xls
../DATA/ENERSHARE_DATA 

,DateTime,AGIA TRIADA,SIDIROKASTRO,KIPI,AGIOI THEODOROI,ALOYMINION,ALOYMINION II,ALOYMINION III,ATHENS,ALEXANDROUPOLIS,...,MOTOR OIL II,XANTHI,OINOFYTA,PLATY,SALFA ANTHOUSSA,SALFA ANO LIOSSIA,SERRES,TRIKALA,Holiday,Weekday
18303,2014-09-01 00:00:00,0,1968.17,1081.71,1.00624,386.378,0,86.9905,107.719,0,...,0,1.93778,90.7407,9.58692,24.8891,28.6224,12.8,0,False,0
18304,2014-09-01 01:00:00,0,1970.4,1085.09,1.42545,88.8906,0,85.5405,97.4709,0,...,0,2.57778,80.108,9.4262,16.2167,28.6496,9.81333,0,False,0
18305,2014-09-01 02:00:00,0,1976.5,1087.03,1.22682,76.0797,0,82.496,85.7418,0,...,0,1.61778,98.6147,9.65596,23.7507,27.3136,1.88444,0,False,0
18306,2014-09-01 03:00:00,0,1981.65,1087.9,1.23624,73.2338,0,82.9419,82.0812,0,...,0,1.20889,102.72,9.69752,1.92702,5.21605,1.45778,0,False,0
18307,2014-09-01 04:00:00,0,1990.28,1088.61,0.997471,71.0866,0,86.5953,84.6462,0,...,0,1.51111,107.171,9.562,0.00578552,6.0069,1.52889,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29243,2018-07-31 19:00:00,2.75239e+06,2.86032e+06,0,2953.97,415547,746906,78550,140348,2647.12,...,743719,4452.5,119194,8751.31,1817.82,1233.17,NaN,NaN,False,1
29244,2018-07-31 20:00:00,2.74082e+06,2.8634e+06,0,1094.55,415838,753553,82596.4,137495,3443.72,...,743089,5420.56,124397,8716.14,10920.8,12353.9,NaN,NaN,False,1
29245,2018-07-31 21:00:00,2.74281e+06,2.87477e+06,0,2347.25,416213,687621,81791.5,136852,2145.74,...,692457,5980,128304,9186.46,11835.5,37350.6,NaN,NaN,False,1
29246,2018-07-31 22:00:00,2.74402e+06,2.89026e+06,0,3245.85,416960,751044,84686.2,140395,1675.96,...,749431,5049.17,129413,8803.36,11526.3,25002.4,NaN,NaN,False,1


### Function that saves a dataframe to atlas cloud

In [ ]:
def save_to_atlas(df):
    """Gets as input a pandas dataframe and saves it to a mongo cloud

    Parameters
    ----------
    df : DataFrame
    
    Returns
    -------
    None
    """

